## Information Systems Project

### Authors
- Suhail Lala,
- Kathryn Mitchell,
- Komla Nyagbe


### Using the Program
The program comes with a console prompt for easy use. Simply run the program and a command line will appear.
To see the different commands and symbols, type `?`. To see documentation on a specific command, type `help <command_name>`.

To determine if a given formula is a tautology, run `formula(<input>)`. For example, `formula('S1,A,(A->(B->C))->((A->B)->(A->C)),S2')` will return:
```
('S1,A,A,A,~A,C,S2') : Fundamental
('S1,A,A,~B,~A,C,S2') : Fundamental
('S1,A,B,A,~A,C,S2') : Fundamental
('S1,A,B,~B,~A,C,S2') : Fundamental
('S1,A,~C,A,~A,C,S2') : Fundamental
('S1,A,~C,~B,~A,C,S2') : Fundamental
Given formula is a Tautology
```

To exit the program, type `exit`.

In [1]:
# To-do: Komla
# Function to transform parenthesis form (a->b)->(~b->~a) to non-parenthesis form ->->ab->~b~a

In [2]:
# Function to split text into Alpha, Beta, S1 and S2
def alpha_beta_split(text):

    # Support for switching between alpha to beta
    alpha_count = 0
    label = 'S1'

    # Dictionary of alpha/beta/S strings
    ab_dict = dict()
    ab_dict['alpha'] = ''
    ab_dict['beta'] = ''
    ab_dict['S1'] = ''
    ab_dict['S2'] = ''

    for i in range(len(text)):  # Loop over every char
        # Switch labels to beta or S2 if conditions are satisfied
        if alpha_count >= 1 and label == 'alpha':
            label = 'beta'
        elif text[i] == ',' and label == 'beta':
            label = 'S2'

        # If current char is a sign
        if text[i] in signs.values():
            if text[i] == signs['negation']:    # Ignore alpha_count for negation
                ab_dict[label] += text[i]       # Add negation to alpha/beta/S
            else:
                if label == 'S1':               # Marks start of alpha
                    label = 'alpha'
                else:
                    ab_dict[label] += text[i]       # Add sign to alpha/beta/S
                    alpha_count -= 1                # Reduce alpha_count to add 1 more atomic statement to alpha

        # If current char is an Atomic statement
        elif text[i] in atomic:
            if label != 'alpha':        # Ignore alpha_count for other labels
                ab_dict[label] += text[i]
            else:
                ab_dict[label] += text[i]
                alpha_count += 1

        # If current char is indecomposable sequence
        else:
            ab_dict[label] += text[i]

    return ab_dict

In [3]:
# Schema [S1,AvB,S2]/[S1,A,B,S2]
def union(text):
    ab_dict = alpha_beta_split(text)
    return ab_dict['S1'] + ab_dict['alpha'] + ',' + ab_dict['beta'] + ab_dict['S2']

In [4]:
# Schema [S1,~(AvB),S2]/[S1,~A,S2;S1,~B,S2]
def neg_union(text):
    ab_dict = alpha_beta_split(text)
    return (ab_dict['S1'] + signs['negation'] + ab_dict['alpha'] + ab_dict['S2'], ab_dict['S1'] + signs['negation'] + ab_dict['beta'] + ab_dict['S2'])

In [5]:
# Schema [S1,(A^B),S2]/[S1,A,S2;S1,B,S2]
def intersection(text):
    ab_dict = alpha_beta_split(text)
    return (ab_dict['S1'] + ab_dict['alpha'] + ab_dict['S2'], ab_dict['S1'] + ab_dict['beta'] + ab_dict['S2'])

In [6]:
# Schema [S1,~~A,S2]/[S1,A,S2]
def double_negation(text):
    ab_dict = alpha_beta_split(text)
    return ab_dict['S1'] + ab_dict['alpha'] + ab_dict['S2']

In [7]:
# Schema [S1,~(A→B),S2]/[S1,A,S2;S1,~B,S2]
def neg_implication(text):
    ab_dict = alpha_beta_split(text)
    return (ab_dict['S1'] + ab_dict['alpha'] + ab_dict['S2'], ab_dict['S1'] + signs['negation'] + ab_dict['beta'] + ab_dict['S2'])

In [8]:
# Schema [S1,~(A^B),S2]/[S1,~A,~B,S2]
def neg_intersection(text):
    ab_dict = alpha_beta_split(text)
    return ab_dict['S1'] + signs['negation'] + ab_dict['alpha'] + ',' + signs['negation'] + ab_dict['beta'] + ab_dict['S2']

In [9]:
# Schema [S1,A→B,S2]/[S1,~A,B,S2]
def implication(text):
    ab_dict = alpha_beta_split(text)
    return ab_dict['S1'] + signs['negation'] + ab_dict['alpha'] + ',' + ab_dict['beta'] + ab_dict['S2']

In [10]:
# Assigns schema and returns output of that schema
def assign_schema(text):
    for i in range(len(text)):
        if text[i] in signs.values():               # Check for signs
            if text[i] == signs['negation']:        # Assign schemas for negative formulas
                if text[i+1] == signs['intersection']:  # Check if next values are signs and assign schema accordingly
                    text = text[:i] + text[i+1:]    # Remove negation sign
                    text = neg_intersection(text)
                    break
                elif text[i+1] == signs['union']:
                    text = text[:i] + text[i+1:]
                    text = neg_union(text)
                    break
                elif text[i+1] == signs['negation']:
                    text = text[:i] + text[i+2:]    # Remove double negation
                    #text = double_negation(text)
                    break
                elif text[i+1] == signs['implication']:
                    text = text[:i] + text[i+1:]
                    text = neg_implication(text)
                    break

            else:                               # Assign schemas for non-negative formulas
                if text[i] == signs['union']: text = union(text)
                elif text[i] == signs['intersection']: text = intersection(text)
                elif text[i] == signs['implication']: text = implication(text)
                break

    # Put string in a tuple to prevent iteration over every character during recursion
    if not isinstance(text, tuple): text = (text,)
    return text

In [11]:
# Helper function to check if a leaf is fundamental
def check_fundamental(leaf_text):
    fundamental = False
    leaf_text = leaf_text.split(',')
    for i in range(len(leaf_text)):
        for j in range(i+1, len(leaf_text)):
            # Check if negation of an atomic statement exists, and if so, assign fundamental = True
            if leaf_text[i] == signs['negation'] + leaf_text[j] or leaf_text[j] == signs['negation'] + leaf_text[i]: fundamental = True
    return fundamental

In [12]:
# Perform depth-first search, output leaf values and if leaf is fundamental
leaf_nodes = []
fundamental = []
def RS_tree(text_input):
    for text_output in assign_schema(text_input):
        if len(fundamental) > 0:                               # Check if fundamental array is empty
            if not fundamental[-1]: break                      # Stop recursion if a leaf is not fundamental

        if text_input != text_output: RS_tree(text_output)     # If leaf has not been reached, perform recursion until leaf is reached
        else:                                                   # If leaf has been reached:
            leaf_nodes.append(text_output)                      # Add leaf node values to array
            fundamental.append(check_fundamental(text_output))  # Add fundamental boolean to array

In [18]:
# Check if formula is Tautology
def check_tautology(text_input):
    tautology = True

    # Reset lists
    global leaf_nodes
    global fundamental
    leaf_nodes = []
    fundamental = []

    RS_tree(text_input)         # Perform Depth first search, populate leaf values and check for fundamental
    for i in range(len(leaf_nodes)):
        print(leaf_nodes[i] + ' : ' + ('Fundamental' if fundamental[i] else 'Not Fundamental')) # Print fundamental value of every leaf
        if fundamental[i] == False: tautology = False       # If a leaf is not fundamental, formula is not tautology
    return tautology

In [14]:
# Initial setup
# Set up a dictionary of signs
signs = dict()
signs['union'] = 'v'
signs['intersection'] = '^'
signs['implication'] = '→'
signs['negation'] = '~'

# Set up list of atomic statements
atomic = []
atomic.append('A')
atomic.append('B')
atomic.append('C')
atomic.append('D')
atomic.append('E')

In [24]:
# console import and setup

from cmd import Cmd
class FormulaPrompt(Cmd):
    intro = "Welcome! Enter a formula or type ? to view commands"
    def do_exit(self, inp):
        '''Exit the command shell'''
        print("Exiting")
        return True
    def do_help(self, inp):
        '''Display help commands'''
        print("S1, S2, S3 etc. denote finite sequences of Indecomposable formulas")
        print("A, B, C, D and E denote Atomic statements")
        print("'v' denotes Union")
        print("'^' denotes Intersection")
        print("'->' denotes Implication")
        print("'~' denotes Negation")
        print("")
        print("To determine if a given formula is a tautology, type formula(<input>). For example, running formula('S1,~A,(A->(B->C))->((A->B)->(A->C)),S2') will return:")
        print('Given formula is a Tautology' if check_tautology("S1,~A,→→A→BC→→AB→AC,S2") else 'Given formula is not a Tautology')
        print("")
        print("To exit the program, type exit")
        print("")
        super(FormulaPrompt, self).do_help(inp)
        pass
    def do_formula(self, inp):
        '''Check if given formula is a tautology'''
        if (inp == ""):
            print("Please enter a formula")
        else:
            inp = inp.replace('->', '→')
            print('Given formula is a Tautology' if check_tautology(inp) else 'Given formula is not a Tautology')
        print("")
        pass
        
FormulaPrompt().cmdloop()

Welcome! Enter a formula or type ? to view commands
S1, S2, S3 etc. denote finite sequences of Indecomposable formulas
A, B, C, D and E denote Atomic statements
'v' denotes Union
'^' denotes Intersection
'->' denotes Implication
'~' denotes Negation

To determine if a given formula is a tautology, type formula(<input>). For example, running formula('S1,~A,(A->(B->C))->((A->B)->(A->C)),S2') will return:
S1,~A,A,A,~A,C,S2 : Fundamental
S1,~A,A,~B,~A,C,S2 : Fundamental
S1,~A,B,A,~A,C,S2 : Fundamental
S1,~A,B,~B,~A,C,S2 : Fundamental
S1,~A,~C,A,~A,C,S2 : Fundamental
S1,~A,~C,~B,~A,C,S2 : Fundamental
Given formula is a Tautology

To exit the program, type exit


Documented commands (type help <topic>):
exit  formula  help

Exiting
